<a href="https://colab.research.google.com/github/MuhammedAshraf2020/NLP-Pytorch/blob/main/Text_Classification_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Data

In [1]:
!mkdir ~/.kaggle
!cp kaggle.json  ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json  

In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 97% 25.0M/25.7M [00:01<00:00, 5.10MB/s]
100% 25.7M/25.7M [00:01<00:00, 17.2MB/s]


In [3]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [8]:
import torch
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer  

In [6]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [7]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Create Our Custom DataLoader

In [67]:
from torch.utils.data import DataLoader , Dataset

In [56]:
def numurize(sent):
  if sent == "positive":
    return 1
  else:
    return 0

In [218]:
dataset = Preprocessing("/content/IMDB Dataset.csv").data

In [221]:
class Sequences(Dataset):
  def __init__(self , path):
    self.data = pd.read_csv(path)
    
    #removing tags , puncs and digits
    self.data["review"] = self.data["review"].str.replace('\d+', '')
    self.data["review"] = self.data["review"].str.replace("<.*?>" , "")
    self.data["review"] = self.data["review"].str.replace("n't"   , " not" )

    #nummirize sntiment/Labels
    self.data["sentiment"] = self.data["sentiment"].apply(lambda x : numurize(x))
    
    #tokinizer and delete stopwords
    self.tokenizer = CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
    self.sequances = self.tokenizer.fit_transform(self.data)
    
    self.labels = self.data["sentiment"].tolist()
    self.token2index = self.tokenizer.vocabulary_
    self.index2token = {idx : token for token , idx in self.token2index.items()}
  
  def __getitem__(self , i):
    return (self.sequances[i].toarray() , self.labels[i])
  
  def __len__(self):
    return self.sequances.shape[0]

In [247]:
dataset_train = Sequences("/content/IMDB Dataset.csv")

In [248]:
train_loader = DataLoader(dataset_train , batch_size = 510)

## Let's Build Model

In [249]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [250]:
class FeedForward(nn.Module):
  def __init__(self , voc_lens):
    super().__init__()
    self.Linear1 = nn.Linear(voc_lens , 128)
    self.Linear2 = nn.Linear(128 , 64)
    self.out = nn.Linear(64 , 1)
  
  def forward(self , x):
    x = x.squeeze(1).float()
    x = F.relu(self.Linear1(x))
    x = F.relu(self.Linear2(x))
    return torch.sigmoid(self.out(x))


In [251]:
model = FeedForward(len(dataset_train.token2index)).to("cuda")

In [252]:
model

FeedForward(
  (Linear1): Linear(in_features=2, out_features=128, bias=True)
  (Linear2): Linear(in_features=128, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)

In [253]:
critic = nn.BCELoss()
opt    = optim.Adam(model.parameters())

## Train / Test Z Model

In [254]:
from tqdm import tqdm , tqdm_notebook

In [255]:
model.train()
epochs_losses = []

for epoch in range(100):
  
  loop   = tqdm_notebook(train_loader , leave = False)
  total  = 0 
  losses = []
  
  for x , y in loop:
    model.zero_grad()
    
    x = x.to("cuda")
    y = y.to("cuda").float()
    
    prediction = model(x)

    loss = critic(prediction.squeeze() , y)
    loss.backward()

    loop.set_description(f"loss : {loss.item():.3f}")

    opt.step()
    losses.append(loss.item())
    total = total + 1
  
  epoch_loss = sum(losses) / total
  epochs_losses.append(epoch_loss)
  
  tqdm.write(f"Epoch {epoch + 1} loss:[{epoch_loss}]")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 loss:[0.7819536328315735]


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 2 loss:[0.7482127547264099]


...

Epoch 98 loss:[0.006794842425733805]


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 99 loss:[0.006651740521192551]


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 100 loss:[0.006513790227472782]


In [256]:
def predict_sentiment(text):
    model.eval()
    with torch.no_grad():
        test_vector = torch.LongTensor(dataset_train.tokenizer.transform([text]).toarray()).to("cuda")

        output = model(test_vector).to("cuda")
        prediction = torch.sigmoid(output).item()

        if prediction > 0.5:
            print(f'{prediction:0.3}: Positive sentiment')
        else:
            print(f'{prediction:0.3}: Negative sentiment')

In [257]:
test_text = """
Cool Cat Saves The Kids is a symbolic masterpiece directed by Derek Savage that
is not only satirical in the way it makes fun of the media and politics, but in
the way in questions as how we humans live life and how society tells us to
live life.

Before I get into those details, I wanna talk about the special effects in this
film. They are ASTONISHING, and it shocks me that Cool Cat Saves The Kids got
snubbed by the Oscars for Best Special Effects. This film makes 2001 look like
garbage, and the directing in this film makes Stanley Kubrick look like the
worst director ever. You know what other film did that? Birdemic: Shock and
Terror. Both of these films are masterpieces, but if I had to choose my
favorite out of the 2, I would have to go with Cool Cat Saves The Kids. It is
now my 10th favorite film of all time.

Now, lets get into the symbolism: So you might be asking yourself, Why is Cool
Cat Orange? Well, I can easily explain. Orange is a color. Orange is also a
fruit, and its a very good fruit. You know what else is good? Good behavior.
What behavior does Cool Cat have? He has good behavior. This cannot be a
coincidence, since cool cat has good behavior in the film.

Now, why is Butch The Bully fat? Well, fat means your wide. You wanna know who
was wide? Hitler. Nuff said this cannot be a coincidence.

Why does Erik Estrada suspect Butch The Bully to be a bully? Well look at it
this way. What color of a shirt was Butchy wearing when he walks into the area?
I don't know, its looks like dark purple/dark blue. Why rhymes with dark? Mark.
Mark is that guy from the Room. The Room is the best movie of all time. What is
the opposite of best? Worst. This is how Erik knew Butch was a bully.

and finally, how come Vivica A. Fox isn't having a successful career after
making Kill Bill.

I actually can't answer that question.

Well thanks for reading my review.
"""
predict_sentiment(test_text)

0.73: Positive sentiment
